In [90]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
from tensorflow.keras.models import load_model
from joblib import load
import joblib
import shutil
from collections import Counter

# Con todos los valores que hay en param se grafican en comparación a las predicciones

Una de graficas reales que grafica todo los datos que se tienen en graficas individuales, luego una que grafica graficas predichas, que predice con el modelo cargado al principio, la tercera compara las 2 pasadas

Carga del modelo normalizado

In [91]:
model = tf.keras.models.load_model('h5/NUEVO2.h5')

# Haciendolo con varias gráficas a la vez

Funcion que genera y guarda imagenes generadas

In [92]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

Limpieza de datos

In [93]:
combinaciones_unicas = param[['Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']].drop_duplicates().reset_index(drop=True)
combinaciones_filtradas = combinaciones_unicas[combinaciones_unicas['Fracvol'] <= 3].reset_index(drop=True)

Creación de las listas que serán graficadas

In [94]:
fracvol = combinaciones_filtradas['Fracvol'].tolist()
radio = combinaciones_filtradas['Radio (nm)'].tolist()
espesor = combinaciones_filtradas['Espesor de la película (nm)'].tolist()
longitud_de_onda = list(range(301, 901))

Borrando carpetas para que no se dupliquen

In [95]:
# Ruta del directorio
dir_path = 'graficas_reales'

# Si el directorio ya existe, eliminarlo
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

# Crear el directorio
os.makedirs(dir_path)

Creación gráficas reales

In [96]:
# Asegurarse de que la carpeta graficas_reales exista
if not os.path.exists('graficas_reales'):
    os.makedirs('graficas_reales')

# Variables para almacenar los datos de AUC y las combinaciones correspondientes
auc_data = []
combinaciones = []

# Filtrar el dataframe para solo trabajar con fracvol = 1
param = param[param['Fracvol'] == 1]

# Obtener las combinaciones únicas y filtradas
combinaciones_unicas = param[['Radio (nm)', 'Espesor de la película (nm)']].drop_duplicates().values

# Iterar sobre las combinaciones únicas y filtradas
for r, e in combinaciones_unicas:
    # Filtrar el dataframe params para obtener datos que coincidan con la combinación actual
    subset = param[(param['Radio (nm)'] == r) & 
                   (param['Espesor de la película (nm)'] == e)]
    
    # Si hay menos de 600 datos para esta combinación, saltar a la siguiente iteración
    if len(subset) < 600:
        continue

    # Calcular el AUC para el rango de 400 a 700 nm
    mask = (subset['Longitud de onda'] >= 400) & (subset['Longitud de onda'] <= 700)
    auc_value = np.trapz(subset['Índice de absorción'][mask], subset['Longitud de onda'][mask])

    # Agregar el valor de AUC y la combinación a las listas correspondientes
    auc_data.append(auc_value)
    combinaciones.append((r, e))

# Convertir los datos a un DataFrame de pandas para facilidad de manejo
df_auc = pd.DataFrame({'AUC': auc_data, 'Combinación': combinaciones})

# Calcular estadísticas descriptivas
mean_auc = df_auc['AUC'].mean()
std_auc = df_auc['AUC'].std()

# Definir las categorías de absorción
bins = pd.cut(df_auc['AUC'], bins=[0, mean_auc - std_auc, mean_auc + std_auc, float('inf')],
              labels=['Poca absorción', 'Mediana absorción', 'Mucha absorción'])

# Asignar las categorías al DataFrame
df_auc['Categoría'] = bins

# Iterar de nuevo para graficar con los títulos actualizados
for idx, (r, e) in enumerate(df_auc['Combinación']):
    # Filtrar el dataframe params para obtener datos que coincidan con la combinación actual
    subset = param[(param['Radio (nm)'] == r) &
                   (param['Espesor de la película (nm)'] == e)]
    
    # Crear la gráfica de dispersión para esta combinación
    plt.figure(figsize=(10, 6))
    plt.plot(subset['Longitud de onda'], subset['Índice de absorción'], color='blue')
    plt.xlabel('Longitud de onda')
    plt.ylabel('Índice de absorción')
    titulo = f'Fracción volumen: 1.00, Radio: {r:.2f}, Espesor: {e:.2f}, AUC (400-700 nm): {df_auc["AUC"][idx]:.2f}, Categoría: {df_auc["Categoría"][idx]}'
    plt.title(titulo)
    plt.grid(True)
    filename = f'graficas_reales/grafica_1.00_{r:.2f}_{e:.2f}.png'
    plt.savefig(filename)
    plt.close()

# Escribir los resultados en un archivo de texto dentro de la carpeta 'graficas_reales'

# Escribir los resultados en un archivo de texto dentro de la carpeta 'graficas_reales'
with open(os.path.join('graficas_reales', 'info_auc_reales.txt'), 'w') as file:
    file.write(f'Máximo AUC: {df_auc["AUC"].max()}, Combinación: (Fracvol: 1, Radio: {df_auc["Combinación"][df_auc["AUC"].idxmax()][0]}, Espesor: {df_auc["Combinación"][df_auc["AUC"].idxmax()][1]})\n')
    file.write(f'Mínimo AUC: {df_auc["AUC"].min()}, Combinación: (Fracvol: 1, Radio: {df_auc["Combinación"][df_auc["AUC"].idxmin()][0]}, Espesor: {df_auc["Combinación"][df_auc["AUC"].idxmin()][1]})\n')
    file.write(f'Media AUC: {mean_auc}\n')
    file.write(f'Desviación Estándar AUC: {std_auc}\n')
    file.write(f'Mediana AUC: {df_auc["AUC"].median()}\n')

Revisa la existencia de la carpeta de gráficas predichas, si existe borra el contenido, si no solo la crea

In [97]:
# Ruta del directorio
dir_path = 'graficas_predichas'

# Si el directorio ya existe, eliminarlo
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

# Crear el directorio
os.makedirs(dir_path)

Predice todos los valores

In [98]:
# Asegurar que la carpeta existe o crearla
carpeta = "graficas_predichas"
if not os.path.exists(carpeta):
    os.mkdir(carpeta)

# Asumiendo que todos los arrays (fracvol, radio, espesor) tienen la misma longitud:
num_elementos = len(fracvol)

scaler_X = load('minmax_scaler_X.pkl')

for i in range(num_elementos):
    data = {
        "Longitud de onda": longitud_de_onda,
        "Fracvol": [fracvol[i]] * len(longitud_de_onda),
        "Radio (nm)": [radio[i]] * len(longitud_de_onda),
        "Espesor de la película (nm)": [espesor[i]] * len(longitud_de_onda)
    }
    
    df = pd.DataFrame(data)
    x = df.values
    x_norm = scaler_X.transform(x)
    predictions = model.predict(x_norm)

    df["Índice de absorción"] = predictions

    columnas_seleccionadas = ['Longitud de onda', 'Índice de absorción']
    df_seleccionado = df[columnas_seleccionadas]
    titulo = 'fracción volumen: {:.2f}, radio: {:.2f}, espesor: {:.2f}'.format(fracvol[i], radio[i], espesor[i])

    # Crear la gráfica de dispersión
    plt.figure(figsize=(10, 6))
    plt.plot(df_seleccionado['Longitud de onda'], df_seleccionado['Índice de absorción'], color='blue')
    plt.xlabel('Longitud de onda')
    plt.ylabel('Índice de absorción')
    plt.title(titulo)
    plt.grid(True)
    
    # Guardar la gráfica como imagen en la carpeta especificada
    nombre_archivo = os.path.join(carpeta, "grafica_{}_{}_{}.png".format(fracvol[i], radio[i], espesor[i]))
    plt.savefig(nombre_archivo)
    plt.close()

19/19 [==============================] - 0s 560us/step


In [99]:
longitud_de_onda = list(range(300, 900))

In [100]:
len(longitud_de_onda)

600

Borrando carpetas si ya existen

In [101]:
# Ruta del directorio
dir_path = 'comparacion_de_datos'

# Si el directorio ya existe, eliminarlo
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)

# Crear el directorio
os.makedirs(dir_path)

# Lista para almacenar los errores MSE
mse_errors = []
# Lista para almacenar la información de cada combinación
combination_info = [] 

In [103]:
param.columns

Index(['Longitud de onda', 'Índice de absorción', 'Fracvol', 'Radio (nm)',
       'Espesor de la película (nm)'],
      dtype='object')

Solo guarda las graficas con errores menores a 2.5611e-06, que fue el error medio cuando tomamos en cuenta todos los valores.
Se está utilizando la regla del trapecio, se divide el area bajo la curva en una serie de trapecios y luego se suma para calcular el total. AUC = area bajo la curva, en un rango de 400 a 700 nm, nos dice cuánta absorción hay. También da el error porcentual del area y genera un archivo txt con la combinación de todos con su error porcentual. 

In [105]:
# Iterate over all possible combinations of fracvol, radio, and espesor
for f, r, e in zip(fracvol, radio, espesor):
    
    # Filter the `param` dataframe to get the real data associated with that combination
    subset = param[(param['Fracvol'] == f) & 
                   (param['Radio (nm)'] == r) & 
                   (param['Espesor de la película (nm)'] == e)]
    
    # If there are less than 600 data points or more than 600, skip to the next iteration
    if len(subset) != 600:
        continue

    # Extract 'Wavelength' and 'Absorption Index'
    X_real = subset[['Longitud de onda', 'Fracvol', 'Radio (nm)', 'Espesor de la película (nm)']].values
    y_real = subset['Índice de absorción'].values

    # Make predictions with the model
    y_pred = model.predict(scaler_X.transform(X_real))

    # Calculate the MSE and add it to the list
    mse = mean_squared_error(y_real, y_pred)
    mse_errors.append(mse)
    y_pred = y_pred.ravel()
    wavelength = np.array(longitud_de_onda)
    
    # Calculate the AUC for the range of 400 to 700 nm
    mask = (X_real[:, 0] >= 400) & (X_real[:, 0] <= 700)
    auc_real = np.trapz(y_real[mask], X_real[mask, 0])
    auc_pred = np.trapz(y_pred[mask], X_real[mask, 0])

    # Calculate the percentage error of the areas
    percentage_error_auc = abs((auc_real - auc_pred) / auc_real) * 100
    
    # Add the combination information to the list
    combination_info.append(f'Volume Fraction: {f}, Radius: {r} nm, Thickness: {e} nm, AUC Percentage Error: {percentage_error_auc:.2f}%\n')
    
    # Save the figure in the "data_comparison" folder if the MSE is below the limit
    if mse <= 2.5547e-06:
        # Plot the real and predicted curves
        plt.figure(figsize=(10, 6))
        plt.plot(wavelength, y_real, label='Real', color='blue')
        plt.plot(wavelength, y_pred, label='Prediction', color='red', linestyle='--')
        plt.xlabel('Wavelength (nm)')
        plt.ylabel('Absorption Index')
        plt.title(f'Real vs Predicted - Volume Fraction: {f}, Radius: {r} nm, Thickness: {e} nm\nMSE: {mse:.4e}, Real AUC: {auc_real:.4e}, Predicted AUC: {auc_pred:.4e}, AUC Error: {percentage_error_auc:.2f}%')
        plt.legend()
        filename = f"Comparison_Fracvol_{f}_Radio_{r}nm_Espesor_{e}nm.png"
        plt.savefig(os.path.join(dir_path, filename))
        plt.close()  # Close the current figure to avoid filling up memory

# Calculate the average of all MSEs
average_mse = sum(mse_errors) / len(mse_errors)

# Create or open the txt file and write the information
with open(os.path.join(dir_path, 'mse_combinations_info.txt'), 'w') as file:
    file.write(f'Average MSE: {average_mse:.4e}\n\n')
    file.write('Combination Information:\n')
    for info in combination_info:
        file.write(info)


19/19 [==============================] - 0s 773us/step


19/19 [==============================] - 0s 844us/step
